## **Подключение библиотек**

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem import rdFingerprintGenerator
import joblib

## **Загрузка модели и данных по которым надо будет сделать "предсказание"**

In [ ]:
model = joblib.load(filename="./sars-cov-2_SI_predictive_model.joblib")
data = pd.read_csv("./input/data.csv")
data

## **Извлечение признаков из `SMILES`**

In [3]:
def extract_morgan_fingerprint(smiles, radius=2, nBits=2048):
    # Преобразуем SMILES в молекулу
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return None
    
    # Создаем генератор Morgan fingerprints
    fpgen = AllChem.GetMorganGenerator(radius=radius, fpSize=nBits)
    
    # Генерируем fingerprint в виде битовой векторной формы
    morgan_fp = fpgen.GetFingerprint(mol)
    
    # Преобразуем результат в массив numpy
    morgan_fp_arr = np.array(morgan_fp)
    
    return morgan_fp_arr

def extract_features(smiles):
    # Преобразование SMILES в молекулу RDKit
    mol = Chem.MolFromSmiles(smiles)
    
    if mol is None:
        return None
    
    # Вычисление молекулярной массы
    molecular_weight = Descriptors.MolWt(mol)
    
    # Вычисление LogP (коэффициента распределения)
    logP = Descriptors.MolLogP(mol)
    
    # Число водородных доноров (HBD)
    hbd = Descriptors.NumHDonors(mol)
    
    # Число водородных акцепторов (HBA)
    hba = Descriptors.NumHAcceptors(mol)
    
    # Топологическая полярная поверхность (TPSA)
    tpsa = Descriptors.TPSA(mol)
    
    rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
    fingerprint = rdkit_gen.GetFingerprint(mol)
    
    

    return {
        'MolecularWeight': molecular_weight,
        'LogP': logP,
        'HBD': hbd,
        'HBA': hba,
        'TPSA': tpsa,
        'fingerprint' : fingerprint
    }

def dataframe_processing(df):
    tmp = df.drop(columns=['IC50 mg/ml', 'CC50 mg/ml', 'SI', 'Cell', 'Virus', 'Strain', 'DOI'], axis=1)
    features_df = tmp['smiles'].apply(extract_features)

    features_expanded_df = pd.DataFrame(features_df.tolist())


    tmp.reset_index(drop=True, inplace=True)
    features_expanded_df.reset_index(drop=True, inplace=True)

    complete_data_with_features = pd.concat([tmp, features_expanded_df], axis=1, ignore_index=False)
    
    
    compounds = []
    for smile, mw, logp, hbd, hba, tpsa in complete_data_with_features[['smiles', 'MolecularWeight', 'LogP', 'HBD', 'HBA', 'TPSA']].itertuples(index=False):
        compounds.append((Chem.MolFromSmiles(smile), mw, logp, hbd, hba, tpsa))
    rdkit_gen = rdFingerprintGenerator.GetRDKitFPGenerator(maxPath=5)
    fingerprints = np.array([rdkit_gen.GetFingerprint(mol) for mol, _, _, _, _, _ in compounds])
    properties = np.array([[mw, logp, hbd, hba, tpsa] for _, mw, logp, hbd, hba, tpsa in compounds])


    combined_data = np.hstack((fingerprints, properties))

    
    
    return combined_data
    
    

In [4]:
X = dataframe_processing(data)
y = model.predict(X)

## **Загрузка результатов в `predictions.csv`**

In [5]:
# Создаем DataFrame из предсказаний
df = pd.DataFrame(y, columns=['SI_prediction'])

# Сохраняем DataFrame в CSV файл
df.to_csv('./output/predictions.csv', index=False)